# Resource usage of the StellarGraph class

> This notebooks records the time and memory (both peak and long-term) required to construct a StellarGraph object for several datasets.

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

This notebook is aimed at helping contributors to the StellarGraph library itself understand how their changes affect the resource usage of the `StellarGraph` object.

Various measures of resource usage for several "real world" graphs of various sizes are recorded:

- time for construction
- memory usage of the final `StellarGraph` object
- peak memory usage during `StellarGraph` construction (both absolute, and additional compared to the raw input data)

These are recorded both with explicit nodes (and node features if they exist), and implicit/inferred nodes.

The memory usage is recorded end-to-end. That is, the recording starts from data on disk and continues until the `StellarGraph` object has been constructed and other data has been cleaned up. This is important for accurately recording the total memory usage, as NumPy arrays can often share data with existing arrays in memory and so retroactive or partial (starting from data in memory) analysis can miss significant amounts of data. The parsing code in `stellargraph.datasets` doesn't allow determining the memory usage of the intermediate nodes and edges input to the `StellarGraph` constructor, and so cannot be used here.

In [1]:
# install StellarGraph if running on Google Colab
import sys
if 'google.colab' in sys.modules:
  %pip install -q stellargraph[demos]==1.1.0b

In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.1.0b")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.1.0b, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

In [3]:
import stellargraph as sg
import pandas as pd
import numpy as np

import gc
import json
import os
import timeit
import tempfile
import tracemalloc

## Optional reddit data

The original GraphSAGE paper evaluated on a reddit dataset, available at <http://snap.stanford.edu/graphsage/#datasets>. This dataset is large (1.3GB compressed) and so there is not automatic download support for it. The following `reddit_path` variable controls whether and how the reddit dataset is included:

- to ignore the dataset: set the variable to `None`
- to include the dataset: download the dataset zip, decompress it, and set the variable to the decompressed directory

In [4]:
reddit_path = os.path.expanduser("~/data/reddit")

## Datasets

### Cora

In [5]:
cora = sg.datasets.Cora()
cora.download()

cora_cites_path = os.path.join(cora.data_directory, "cora.cites")
cora_content_path = os.path.join(cora.data_directory, "cora.content")
cora_dtypes = {0: int, **{i: np.float32 for i in range(1, 1433 + 1)}}


def cora_parts(include_nodes):
    if include_nodes:
        nodes = pd.read_csv(
            cora_content_path,
            header=None,
            sep="\t",
            index_col=0,
            usecols=range(0, 1433 + 1),
            dtype=cora_dtypes,
            na_filter=False,
        )
    else:
        nodes = None
    edges = pd.read_csv(
        cora_cites_path,
        header=None,
        sep="\t",
        names=["source", "target"],
        dtype=int,
        na_filter=False,
    )
    return nodes, edges, {}

### FB15k

In [6]:
fb15k = sg.datasets.FB15k()
fb15k.download()
fb15k_files = [
    os.path.join(fb15k.data_directory, f"freebase_mtr100_mte100-{x}.txt")
    for x in ["train", "test", "valid"]
]


def fb15k_parts(include_nodes, usecols=None):
    loaded = [
        pd.read_csv(
            name,
            header=None,
            names=["source", "label", "target"],
            sep="\t",
            dtype=str,
            na_filter=False,
            usecols=usecols,
        )
        for name in fb15k_files
    ]
    edges = pd.concat(loaded, ignore_index=True)

    if include_nodes:
        # infer the set of nodes manually, in a memory-minimal way
        raw_nodes = set(edges.source)
        raw_nodes.update(edges.target)
        nodes = pd.DataFrame(index=raw_nodes)
    else:
        nodes = None

    return nodes, edges, {"edge_type_column": "label"}


def fb15k_no_labels_parts(include_nodes):
    nodes, edges, _ = fb15k_parts(include_nodes, usecols=["source", "target"])
    return nodes, edges, {}

### Reddit

As discussed above, the reddit dataset is large and optional. It is also slow to parse, as the graph structure is a huge JSON file. Thus, we prepare the dataset by converting that JSON file into a NumPy edge list array, of shape `(num_edges, 2)`. This is significantly faster to load from disk.

In [7]:
%%time

# if requested, prepare the reddit dataset by saving the slow-to-read JSON to a temporary .npy file
if reddit_path is not None:
    reddit_graph_path = os.path.join(reddit_path, "reddit-G.json")
    reddit_feats_path = os.path.join(reddit_path, "reddit-feats.npy")

    with open(reddit_graph_path) as f:
        reddit_g = json.load(f)
    reddit_numpy_edges = np.array([[x["source"], x["target"]] for x in reddit_g["links"]])
    
    reddit_edges_file = tempfile.NamedTemporaryFile(suffix=".npy")
    np.save(reddit_edges_file, reddit_numpy_edges)

CPU times: user 17.9 s, sys: 2.08 s, total: 19.9 s
Wall time: 20 s


In [8]:
def reddit_parts(include_nodes):
    if include_nodes:
        raw_nodes = np.load(reddit_feats_path)
        nodes = pd.DataFrame(raw_nodes)
    else:
        nodes = None

    raw_edges = np.load(reddit_edges_file.name)
    edges = pd.DataFrame(raw_edges, columns=["source", "target"])
    return nodes, edges, {}

### Collected

In [9]:
datasets = {
    "Cora": cora_parts,
    "FB15k (no labels)": fb15k_no_labels_parts,
    "FB15k": fb15k_parts,
}
if reddit_path is not None:
    datasets["reddit"] = reddit_parts

## Measurement

In [10]:
def mem_snapshot_diff(after, before):
    """Total memory difference between two tracemalloc.snapshot objects"""
    return sum(elem.size_diff for elem in after.compare_to(before, "lineno"))

In [11]:
# names of columns computed by the measurement code
def measurement_columns(title):
    names = [
        "time",
        "mem (SG)",
        "peak mem (SG)",
        "peak mem (SG above data)",
        "mem (data)",
        "peak mem (data)",
    ]
    return [(title, x) for x in names]


columns = pd.MultiIndex.from_tuples(
    [
        ("graph", "nodes"),
        ("graph", "node feat size"),
        ("graph", "edges"),
        *measurement_columns("explicit nodes"),
        *measurement_columns("inferred nodes (no features)"),
    ]
)
columns

MultiIndex([(                       'graph',                    'nodes'),
            (                       'graph',           'node feat size'),
            (                       'graph',                    'edges'),
            (              'explicit nodes',                     'time'),
            (              'explicit nodes',                 'mem (SG)'),
            (              'explicit nodes',            'peak mem (SG)'),
            (              'explicit nodes', 'peak mem (SG above data)'),
            (              'explicit nodes',               'mem (data)'),
            (              'explicit nodes',          'peak mem (data)'),
            ('inferred nodes (no features)',                     'time'),
            ('inferred nodes (no features)',                 'mem (SG)'),
            ('inferred nodes (no features)',            'peak mem (SG)'),
            ('inferred nodes (no features)', 'peak mem (SG above data)'),
            ('inferred nodes (no featu

In [12]:
def measure_time(f, include_nodes):
    nodes, edges, args = f(include_nodes)
    start = timeit.default_timer()
    sg.StellarGraph(nodes, edges, **args)
    end = timeit.default_timer()
    return end - start

In [13]:
def measure_memory(f, include_nodes):
    """
    Measure exactly what it takes to load the data.
    
    - the size of the original edge data (as a baseline)
    - the size of the final graph
    - the peak memory use of both
    
    This uses a similar technique to the 'allocation_benchmark' fixture in tests/test_utils/alloc.py.
    """
    gc.collect()

    tracemalloc.start()
    snapshot_start = tracemalloc.take_snapshot()

    nodes, edges, args = f(include_nodes)

    gc.collect()
    _, data_memory_peak = tracemalloc.get_traced_memory()
    snapshot_data = tracemalloc.take_snapshot()

    if include_nodes:
        assert nodes is not None, f
        sg_g = sg.StellarGraph(nodes, edges, **args)
    else:
        assert nodes is None, f
        sg_g = sg.StellarGraph(edges=edges, **args)

    # clean up the input data and anything else leftover, so that the snapshot
    # includes only the long-lasting data: the StellarGraph.
    del edges
    del nodes
    del args
    gc.collect()

    _, graph_memory_peak = tracemalloc.get_traced_memory()
    snapshot_end = tracemalloc.take_snapshot()
    tracemalloc.stop()

    data_memory = mem_snapshot_diff(snapshot_data, snapshot_start)
    graph_memory = mem_snapshot_diff(snapshot_end, snapshot_start)

    return (
        sg_g,
        graph_memory,
        graph_memory_peak,
        graph_memory_peak - data_memory,
        data_memory,
        data_memory_peak,
    )

In [14]:
def measure(f):
    time_nodes = measure_time(f, include_nodes=True)
    time_no_nodes = measure_time(f, include_nodes=False)

    sg_g, *mem_nodes = measure_memory(f, include_nodes=True)
    _, *mem_no_nodes = measure_memory(f, include_nodes=False)

    return [
        sg_g.number_of_nodes(),
        sg_g.node_feature_sizes()[sg_g.unique_node_type()],
        sg_g.number_of_edges(),
        time_nodes,
        *mem_nodes,
        time_no_nodes,
        *mem_no_nodes,
    ]

In [15]:
%%time
recorded = [measure(f) for f in datasets.values()]

CPU times: user 22.9 s, sys: 5.27 s, total: 28.2 s
Wall time: 28.2 s


In [16]:
raw = pd.DataFrame(recorded, columns=columns, index=datasets.keys())
raw

graph                          explicit nodes             \
                    nodes node feat size     edges           time   mem (SG)   
Cora                 2708           1433      5429       0.026358   15607020   
FB15k (no labels)   14951              0    592213       0.141559    6398752   
FB15k               14951              0    592213       0.857859   12109397   
reddit             232965            602  11606919       4.646127  712113351   

                                                                      \
                  peak mem (SG) peak mem (SG above data)  mem (data)   
Cora                   46772250                 31088362    15683888   
FB15k (no labels)      37222062                 21542696    15679366   
FB15k                  57288365                 36386291    20902074   
reddit               3551638965               2243953025  1307685940   

                                  inferred nodes (no features)             \
                  peak mem (data)                         time   mem (SG)   
Cora                     31993944                     0.007129      94284   
FB15k (no labels)        25831073                     0.232367    6398736   
FB15k                    35791845                     0.895807   12072501   
reddit                 1307694919                     1.097318  153909433   

                                                                     \
                  peak mem (SG) peak mem (SG above data) mem (data)   
Cora                     405221                   314961      90260   
FB15k (no labels)      37223438                 21669584   15553854   
FB15k                  57252173                 36475611   20776562   
reddit                624260822                438546030  185714792   

                                   
                  peak mem (data)  
Cora                       197784  
FB15k (no labels)        25050234  
FB15k                    35011006  
reddit                  185723195

## Pretty results

This shows the results in a prettier way, such as memory in MiB instead of bytes.

In [17]:
mem_columns = raw.columns[["mem" in x[1] for x in raw.columns]]

memory_mib = raw.copy()
memory_mib[mem_columns] /= 2 ** 20
memory_mib

graph                          explicit nodes              \
                    nodes node feat size     edges           time    mem (SG)   
Cora                 2708           1433      5429       0.026358   14.884014   
FB15k (no labels)   14951              0    592213       0.141559    6.102325   
FB15k               14951              0    592213       0.857859   11.548421   
reddit             232965            602  11606919       4.646127  679.124213   

                                                                       \
                  peak mem (SG) peak mem (SG above data)   mem (data)   
Cora                  44.605494                29.648172    14.957321   
FB15k (no labels)     35.497725                20.544716    14.953009   
FB15k                 54.634442                34.700671    19.933771   
reddit              3387.106862              2140.000367  1247.106495   

                                  inferred nodes (no features)              \
                  peak mem (data)                         time    mem (SG)   
Cora                    30.511803                     0.007129    0.089916   
FB15k (no labels)       24.634431                     0.232367    6.102310   
FB15k                   34.133763                     0.895807   11.513234   
reddit                1247.115058                     1.097318  146.779473   

                                                                      \
                  peak mem (SG) peak mem (SG above data)  mem (data)   
Cora                   0.386449                 0.300370    0.086079   
FB15k (no labels)     35.499037                20.665726   14.833311   
FB15k                 54.599927                34.785853   19.814074   
reddit               595.341513               418.230085  177.111427   

                                   
                  peak mem (data)  
Cora                     0.188622  
FB15k (no labels)       23.889765  
FB15k                   33.389097  
reddit                 177.119441

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/internal-developers/graph-resource-usage.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>